# HCP Retinotopy Data Loader

The HCP 7T retinotopyy dataset comprises fMRI retinotopy from 183 human subjects. The NMA-curated dataset includes the average data over all those subjects.

In order to use this dataset, please electronically sign the HCP data use terms at [ConnectomeDB](https://db.humanconnectome.org). Instructions for this are on pp. 24-25 of the [HCP Reference Manual](https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP_S1200_Release_Reference_Manual.pdf).

The data and experiment are decribed in detail in [Benson et al.](https://jov.arvojournals.org/article.aspx?articleid=2719988#207329261)

In [ ]:
#!pip install nilearn --quiet
#!pip install nltools --quiet
#!pip install git+https://github.com/VU-Cog-Sci/nideconv --quiet
!pip install matplotlib==3.3.0

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd

from nilearn import plotting
from scipy.stats import gamma
from matplotlib import animation
from nltools.data import Design_Matrix

# Basic parameters

In [ ]:
# The download cells will store the data in nested directories starting here:
DATA_DIR = "./hcp_retino"
if not os.path.isdir(DATA_DIR):
  os.mkdir(DATA_DIR)

# The data acquisition rate
TR = 1  # Time resolution, in sec

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
RUN_NAMES = [
  "BAR1",   # Sweeping Bars repeat 1
  "BAR2",   # Sweeping Bars repeat 2
  "CCW",    # Counter Clockwise rotating wedge
  "CW",     # Clockwise rotating wedge
  "EXP",    # Expanding ring
  "CON"     # Contracting ring
]

# Downloading data

The files that we provide are:

 - The whole brain time series data in [CIFTI](https://nipy.org/nibabel/reference/nibabel.cifti2.html#module-nibabel.cifti2.cifti2) format
 - The stimulus image files as numpy arrays (time x height x width x 3)
 - surface files for visualisation on the brain

In [ ]:
# @title Download the data

import os, requests, tarfile

fname = "hcp_retino.tgz"
url = "https://osf.io/d25b4/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      print(f"Downloading {fname}...")
      with open(fname, "wb") as fid:
        fid.write(r.content)
      print(f"Download {fname} completed!")

In [ ]:
# @title Extract the data in `DATA_DIR`
fname_ex = "HCP7T_retino"
path_name = os.path.join(DATA_DIR, fname_ex)
if not os.path.exists(path_name):
  print(f"Extracting {fname}...")
  with tarfile.open(f"{fname}") as fzip:
    fzip.extractall(DATA_DIR)
else:
  print(f"File {fname}.tgz has already been extracted.")



# Helper functions

## load all the data

In [ ]:
#load all the data

run_exp = 'BAR2'
filename_signal = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_PA.dtseries.nii')
cifti = nib.load(filename_signal)
time_series_bar2 = cifti.get_fdata()

run_exp = 'EXP'
filename_signal = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_AP.dtseries.nii')
cifti = nib.load(filename_signal)
time_series_exp = cifti.get_fdata()

run_exp = 'BAR1'
filename_signal = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_AP.dtseries.nii')
cifti = nib.load(filename_signal)
time_series_bar1 = cifti.get_fdata()

run_exp = 'CON'
filename_signal = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_PA.dtseries.nii')
cifti = nib.load(filename_signal)
time_series_con = cifti.get_fdata()

run_exp = 'CW'
filename_signal = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_PA.dtseries.nii')
cifti = nib.load(filename_signal)
time_series_cw = cifti.get_fdata()

run_exp = 'CCW'
filename_signal = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_AP.dtseries.nii')
cifti = nib.load(filename_signal)
time_series_ccw = cifti.get_fdata()


In [ ]:
def surf_data_from_cifti(data, cifti, surf_name):
  """Maps data from cifti file onto a surface

  Args:
      data : 2D array (nvertices x p)
      cifti : Cifti2Image object
      surf_name : str (either 'LEFT' or 'RIGHT')
  """
  surf_name_long = 'CIFTI_STRUCTURE_CORTEX_'+surf_name
  axis = cifti.header.get_axis(1)  # The brain axis
  for name, data_indices, model in axis.iter_structures():
    if name == surf_name_long:
      vtx_indices = model.vertex
      surf_data = np.zeros(axis.nvertices[surf_name_long])
      surf_data[vtx_indices] = data[data_indices]
      return surf_data
  raise ValueError(f"No structure named {surf_name_long}")

## old hrf

In [ ]:
# HRF FUNCTION TO MODEL BOLD RESPONSE TO A STIMULUS AT T = 0

from scipy.stats import gamma

def hrf(t):
     """ Return values for HRF at given times """

     # Gamma pdf for the peak
     peak_values = gamma.pdf(t, 5.7) # peak at about 5 sec

     # Gamma pdf for the undershoot
     undershoot_values = gamma.pdf(t, 12) # min at about 12 sec

     # Combine them
     values = peak_values - undershoot_values

     # Scale min
     values = values / np.min(values) * -0.01 #****work on this

     # Scale max
     values = values / np.max(values) * 0.5

     # return BOLD signal values
     return values

hrf_times = np.arange(0, 25, 1)
hrf_signal = hrf(hrf_times)
plt.plot(hrf_times, hrf_signal)
plt.axhline(y=0, color='r', linestyle='-')
plt.xlabel('time (seconds)')
plt.ylabel('BOLD signal')
plt.title('Estimated BOLD signal for event at time 0')

## stacking the data

In [ ]:
# organize and label the data and stimuli
# 1 = exp
time_series_exp_demean = np.array(time_series_exp)
for i in range(91282):
  time_series_exp_demean[:, i] -= np.mean(time_series_exp_demean[:, i])
time_series_stims = time_series_exp_demean
stim_on_exp = stim_area_exp
stim_on_exp[stim_on_exp>0] = 1
stims_on = stim_on_exp
# 2 = con
time_series_con_demean = np.array(time_series_con)
for i in range(91282):
  time_series_con_demean[:, i] -= np.mean(time_series_con_demean[:, i])
time_series_stims = np.append(time_series_stims, np.array(time_series_con_demean), 0)
stims_on = np.append(stims_on, stim_on_exp*2)
# 3 = CW
time_series_cw_demean = np.array(time_series_cw)
for i in range(91282):
  time_series_cw_demean[:, i] -= np.mean(time_series_cw_demean[:, i])
time_series_stims = np.append(time_series_stims, np.array(time_series_cw_demean), 0)
stim_on_cw = np.append(np.zeros(22), np.append(np.ones(256)*3,np.zeros(22)))
stims_on = np.append(stims_on, stim_on_cw)
# 4 = BAR1
time_series_bar1_demean = np.array(time_series_bar1)
for i in range(91282):
  time_series_bar1_demean[:, i] -= np.mean(time_series_bar1_demean[:, i])
time_series_stims = np.append(time_series_stims, np.array(time_series_bar1_demean), 0)
stim_on_bar1 = bar_direction
stim_on_bar1[stim_on_bar1>0] = 4
stims_on = np.append(stims_on, stim_on_bar1)
# 5 = CCW
time_series_ccw_demean = np.array(time_series_ccw)
for i in range(91282):
  time_series_ccw_demean[:, i] -= np.mean(time_series_ccw_demean[:, i])
time_series_stims = np.append(time_series_stims, np.array(time_series_ccw_demean), 0)
stim_on_cw = np.append(np.zeros(22), np.append(np.ones(256)*5,np.zeros(22)))
stims_on = np.append(stims_on, stim_on_ccw)
# 6 = BAR2
'''time_series_bar2_demean = np.array(time_series_bar2)
for i in range(91282):
  time_series_bar2_demean[:, i] -= np.mean(time_series_bar2_demean[:, i])
time_series_stims = np.append(time_series_stims, np.array(time_series_bar2_demean), 0)
stim_on_bar2 = bar_direction
stim_on_bar2[stim_on_bar2>0] = 6
stims_on = np.append(stims_on, stim_on_bar2)'''

## voxel specific hrf estimate

In [ ]:
#estimating HRF for every voxel
import nideconv as nd

rf = nd.ResponseFitter(input_signal=time_series_stims,
                            sample_rate=1)
stims = [idx for idx, val in enumerate(stims_on) if val != 0]
n_regressors = 25

rf.add_event(event_name='stim',
             onsets=stims,
             basis_set='fourier',
             n_regressors=n_regressors,
             interval=[0,25])

rf.fit()

hrf_poor = rf.events['stim'].get_timecourses()
hrf_est = np.array(hrf_poor)
hrf_est.shape

# Expanding circle (continuous stim - deprecated)

Import time series (BOLD signal) data

In [ ]:
## TIME SERIES DATA

# pick a run
run_exp = 'EXP'

# define filename
filename_signal_exp = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_AP.dtseries.nii')

# load the file with nibabel
cifti_exp = nib.load(filename_signal_exp)

time_series_exp = cifti_exp.get_fdata()
time_series_exp.shape

Import & visualize EXP stimulus data

In [ ]:
# define filename
filename_stim_exp = os.path.join(DATA_DIR, fname_ex, f'stims/7T_RET{run_exp}.npy')

# load the numpy array for the stimulus
stim_exp = np.load(filename_stim_exp)

# check the shape
stim_exp.shape # time x Y (height) x X (width) x 3

# define stimulus cycle
n_before = 22
period = 32
cycles = 8
n_after = 22

def frame(t): # n is frame number

    global stim_exp
    stim_exp_tmp = stim_exp[t, :, :, :]

    plot=plt.imshow(stim_exp_tmp)
    return plot

fig = plt.figure(figsize=(8,6))
stim_frames = np.arange(n_before, n_before + period + 1) # one cycle (running all 300 takes a lot of time)
anim = animation.FuncAnimation(fig, frame, frames = stim_frames, blit = False, repeat = True)

from IPython.display import HTML
HTML(anim.to_html5_video())

In [ ]:
# DEFINE STIMULUS FUNCTION FOR EXPANDING CIRCLE

outer_cycle = np.arange(1, 29)
inner_cycle = np.concatenate([[0, 0, 0, 0, 0, 0], np.arange(1, 23)])
outer_cycle[22:] = 22
outer_radius = np.zeros(300)
inner_radius = np.zeros(300)

for i in range(8):
  outer_radius[(22+i*32):(50+i*32)] = outer_cycle
  inner_radius[(22+i*32):(50+i*32)] = inner_cycle

plt.plot(np.pi*outer_radius**2-np.pi*inner_radius**2)

stim_area = np.pi*outer_radius**2-np.pi*inner_radius**2

Convolve HRF & stimulus to get expected response

In [ ]:
# GLM MODEL
constant_exp = np.ones_like(stim_area)
X_exp = np.column_stack([constant_exp, (np.convolve(stim_area, hrf_signal)[:300])])

theta_hats_exp = np.linalg.inv(X_exp.T @ X_exp) @ X_exp.T @ time_series_exp

predicted_time_series_exp = X_exp @ theta_hats_exp

In [ ]:
# PLOT THETA WEIGHTS ON BRAIN
filename = "Q1-Q6_R440.L.inflated.32k_fs_LR.surf.gii"
pathf = os.path.join(DATA_DIR, fname_ex, "surf", filename)
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(theta_hats_exp[1, :], cifti_exp, 'LEFT'),
                   cmap = 'hsv', threshold = .0001)

# CW rotating wedge (continuous stim - deprecated)

In [ ]:
## LOAD TIME SERIES DATA

# pick a run
run_cw = 'CW'

# define filename
filename_signal_cw = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_cw}_7T_PA.dtseries.nii')

# load the file with nibabel
cifti_cw = nib.load(filename_signal_cw)

# extract time series from file
time_series_cw = cifti_cw.get_fdata()

In [ ]:
## LOAD STIM DATA

# define filename
filename_stim_cw = os.path.join(DATA_DIR, fname_ex, f'stims/7T_RET{run_cw}.npy')

# load the numpy array for the stimulus
stim_cw = np.load(filename_stim_cw)

# check the shape
stim_cw.shape # time x Y (height) x X (width) x 3

# define stimulus cycle
n_before = 22
period = 32
cycles = 8
n_after = 22

In [ ]:
# plot stimulus

def frame(t): # n is frame number

    global stim_cw
    stim_cw_tmp = stim_cw[t, :, :, :]

    plot=plt.imshow(stim_cw_tmp)
    return plot

fig = plt.figure(figsize=(8,6))
stim_frames = np.arange(n_before, n_before + period + 1) # one cycle (running all 300 takes a lot of time)
anim = animation.FuncAnimation(fig, frame, frames = stim_frames, blit = False, repeat = True)

from IPython.display import HTML
HTML(anim.to_html5_video())

In [ ]:
## DEFINE STIMULUS FUNCTION FOR CW ROTATING WEDGE (angle with respect to origin lines)

import math

# FIND INITIAL ANGLE

# get circle center
center_y = len(stim_cw[0, :, 0]) / 2 # find y-coord of wedge center
center_x = len(stim_cw[0, 0, :]) / 2 # find x-coord of wedge center

# find the x-coord of where the white starts at y = 0
top_row = np.sum(stim_cw, axis = 3)[22, 0, :]
x2 = np.where(top_row == np.amax(top_row))[0][0]

# plot
plt.imshow(stim_cw[22, :, :, :]) # plot initial frame
plt.plot(center_x, center_y, 'ro') # draw point at center
plt.plot([center_x, center_x], [center_y, 0], 'r--') # draw straight vertical line at the x-coord of the center
plt.plot(x2, 0, 'ro') # draw point where wedge begins
plt.plot([center_x, x2], [center_y, 0], 'r--') # draw line from center to where wedge begins

# get angle of right triangle (treat center as (0,0))
v1 = np.array([0, center_y])
v2 = np.array([x2 - center_x, center_y])

unit_v1 = v1 / np.linalg.norm(v1)
unit_v2 = v2 / np.linalg.norm(v2)

initial_angle = np.arccos(np.clip(np.dot(unit_v1, unit_v2), -1.0, 1.0)) # radians

print(initial_angle)

In [ ]:
# ADD RADIANS AND FIND COS AS WE ROTATE
plt.rcParams['figure.figsize'] = [8, 6]

rate = 2*np.pi / 32 # rad/s
stim_time = 300 - n_before - n_after
angle = np.zeros(stim_time)
cosine = np.zeros(stim_time)

for i in range(stim_time):

  angle[i] = initial_angle + rate*i
  cosine[i] = math.cos(angle[i])

# cosine = np.concatenate(([np.nan] * n_before, cosine, [np.nan] * n_after), axis = None) # no cosine before or after stimulus appears
plt.plot(cosine)
plt.title("Angle of the rotating wedge")
# plt.xlim([0, 300])
plt.show()

In [ ]:
## RADIANS AS STIMULUS FUNCTION
n_before = 22
n_after = 22
period = 32
rate = 2*np.pi / period # rad/s
stim_time = 300 - n_before - n_after

radians = np.arange(start = initial_angle, stop = initial_angle + 2*np.pi, step = rate)
angle = np.tile(radians, 8)

plt.plot(angle)
# plt.xlim([0, 300])
plt.show()

In [ ]:
# GLM MODEL
constant = np.ones_like(cosine)
X_cw = np.column_stack([constant, (np.convolve(cosine, hrf_signal)[:len(cosine)])])

theta_hats_cw = np.linalg.inv(X_cw.T @ X_cw) @ X_cw.T @ time_series_cw[n_before:(300 - n_after)]

predicted_time_series_cw = X_cw @ theta_hats_cw

In [ ]:
plt.plot(np.convolve(angle, hrf_signal)[:len(angle)])

In [ ]:
plt.plot(np.convolve(cosine, hrf_signal)[:len(cosine)])

In [ ]:
# PLOT THETA WEIGHTS ON BRAIN
filename = "Q1-Q6_R440.L.inflated.32k_fs_LR.surf.gii"
pathf = os.path.join(DATA_DIR, fname_ex, "surf", filename)
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(theta_hats_cw[1, :], cifti_cw,'LEFT'),
                   cmap = 'hsv', threshold = .0001)

**CCW stimulus:**

In [ ]:
# pick a run
run_cw = 'CCW'

# define filename
filename_signal_exp = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_cw}_7T_AP.dtseries.nii')

# load the file with nibabel
cifti_exp = nib.load(filename_signal_exp)

time_series_exp = cifti_exp.get_fdata()
time_series_exp.shape

In [ ]:
## LOAD STIM DATA

# define filename
filename_stim_cw = os.path.join(DATA_DIR, fname_ex, f'stims/7T_RET{run_cw}.npy')

# load the numpy array for the stimulus
stim_cw = np.load(filename_stim_cw)

# check the shape
stim_cw.shape # time x Y (height) x X (width) x 3

# define stimulus cycle
n_before = 22
period = 32
cycles = 8
n_after = 22

In [ ]:
# plot stimulus

def frame(t): # n is frame number

    global stim_cw
    stim_cw_tmp = stim_cw[t, :, :, :]

    plot=plt.imshow(stim_cw_tmp)
    return plot

fig = plt.figure(figsize=(8,6))
stim_frames = np.arange(n_before, n_before + period + 1) # one cycle (running all 300 takes a lot of time)
anim = animation.FuncAnimation(fig, frame, frames = stim_frames, blit = False, repeat = True)

from IPython.display import HTML
HTML(anim.to_html5_video())

In [ ]:
## DEFINE STIMULUS FUNCTION FOR CW ROTATING WEDGE (angle with respect to origin lines)

import math
import cv2
# FIND INITIAL ANGLE

# get circle center
center_y = len(stim_cw[0, :, 0]) / 2 # find y-coord of wedge center
center_x = len(stim_cw[0, 0, :]) / 2 # find x-coord of wedge center

#Load the first frame at time 22s:
im = stim_cw[22, :, :, :]
#create grey image (we cant create a B&W image directly):
grayImage = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#create full black and white image:
(thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 10, 255, cv2.THRESH_BINARY)
#Get all the X and Y coordinates where the coulour is white:
(Xw, Yw) = np.where(blackAndWhiteImage==255)
#Get the x coordinate where the white starts:
x2 = np.amax(Xw)

# plot
plt.imshow(stim_cw[22, :, :, :]) # plot initial frame
plt.plot(center_x, center_y, 'ro') # draw point at center
plt.plot([center_x, center_x], [center_y, 0], 'r--') # draw straight vertical line at the x-coord of the center
plt.plot(x2, 0, 'ro') # draw point where wedge begins
plt.plot([center_x, x2], [center_y, 0], 'r--') # draw line from center to where wedge begins

# get angle of right triangle (treat center as (0,0))
v1 = np.array([0, center_y])
v2 = np.array([x2 - center_x, center_y])

unit_v1 = v1 / np.linalg.norm(v1)
unit_v2 = v2 / np.linalg.norm(v2)

initial_angle = np.arccos(np.clip(np.dot(unit_v1, unit_v2), -1.0, 1.0)) # radians

print (initial_angle)


In [ ]:
# ADD RADIANS AND FIND COS AS WE ROTATE
plt.rcParams['figure.figsize'] = [8, 6]

rate = 2*np.pi / 32 # rad/s
stim_time = 300 - n_before - n_after
angle = np.zeros(stim_time)
cosine = np.zeros(stim_time)

for i in range(stim_time):

  angle[i] = initial_angle + rate*i
  cosine[i] = math.cos(angle[i])

# cosine = np.concatenate(([np.nan] * n_before, cosine, [np.nan] * n_after), axis = None) # no cosine before or after stimulus appears
plt.plot(cosine)
plt.title("Angle of the rotating wedge")
# plt.xlim([0, 300])
plt.show()

In [ ]:
## RADIANS AS STIMULUS FUNCTION
n_before = 22
n_after = 22
period = 32
rate = 2*np.pi / period # rad/s
stim_time = 300 - n_before - n_after

radians = np.arange(start = initial_angle, stop = initial_angle + 2*np.pi, step = rate)
angle = np.tile(radians, 8)

plt.plot(angle)
# plt.xlim([0, 300])
plt.show()

# Moving bars (continuous stim - deprecated)

In [ ]:
## LOAD TIME SERIES DATA

# pick a run
run_bar1 = 'BAR1'

# define filename
filename_signal_bar1 = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_bar1}_7T_AP.dtseries.nii')

# load the file with nibabel
cifti_bar1 = nib.load(filename_signal_bar1)

# extract time series from file
time_series_bar1 = cifti_bar1.get_fdata()

In [ ]:
## LOAD STIM DATA

# define filename
filename_bar1 = os.path.join(DATA_DIR, fname_ex, f'stims/7T_RET{run_bar1}.npy')

# load the numpy array for the stimulus
stim_bar1 = np.load(filename_bar1)

# check the shape
stim_bar1.shape # time x Y (height) x X (width) x 3

# define stimulus cycle
n_before = 16
period = 28
cycles = 8
n_after = 20

def frame(t): # n is frame number

    global stim_bar1
    stim_bar1_tmp = stim_bar1[t, :, :, :]

    plot=plt.imshow(stim_bar1_tmp)
    return plot

fig = plt.figure(figsize=(8,6))
stim_frames = np.arange(142+n_before, 206+n_before + period + 1) # one cycle (running all 300 takes a lot of time)
anim = animation.FuncAnimation(fig, frame, frames = stim_frames, blit = False, repeat = True)

from IPython.display import HTML
HTML(anim.to_html5_video())

In [ ]:
#using the direction of movement, and position along that direction as parameters

#position of the moving bar
bar_area = np.sum(np.sum(np.sum(stim_bar1, 1), 1), 1) #area of the bar
bar_position = bar_area
bar_position[bar_position>0] = 1 # periods when bar is present
for i in range(299):
  if (bar_position[i+1] > 0 and bar_position[i]>0): bar_position[i+1] = bar_position[i] + 1 #constant bar movement along the direction

#direction of movement: L-R, B-T, R-L, T-B, BL-TR, BR-TL, TR-BL, TL-BR
bar_direction = np.zeros((300))
bar_directions = [90, 360, 270, 180, 45, 315, 225, 135] #assuming 0deg is pointing up and going clockwise
bar_periods = [np.arange(16, (16+28)), np.arange((1*32+16), (1*32+16+28)), np.arange((2*32+16),(2*32+16+28)), np.arange((3*32+16),(3*32+16+28)), np.arange((142+0*32+16),(142+0*32+16+28)), np.arange((142+1*32+16),(142+1*32+16+28)), np.arange((142+2*32+16),(142+2*32+16+28)), np.arange((142+3*32+16),(142+3*32+16+28))]
for i in range(8):
  bar_direction[bar_periods[i]] = bar_directions[i]
bar_cos = np.cos(bar_direction/180*np.pi)
plt.title("Position along the direction")
plt.plot(bar_direction)

In [ ]:
#GLM model
constant = np.ones_like(bar_direction)
X_bar1 = np.column_stack([constant, (np.convolve(bar_direction, hrf_signal)[:len(bar_direction)]), (np.convolve(bar_position, hrf_signal)[:len(bar_position)])])

theta_hats_bar1 = np.linalg.inv(X_bar1.T @ X_bar1) @ X_bar1.T @ time_series_bar1

predicted_time_series_cw = X_bar1 @ theta_hats_bar1

In [ ]:
# PLOT THETA WEIGHTS ON BRAIN
filename = "Q1-Q6_R440.L.inflated.32k_fs_LR.surf.gii"
pathf = os.path.join(DATA_DIR, fname_ex, "surf", filename)
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(theta_hats_bar1[1, :], cifti,'LEFT'),
                   cmap = 'hsv', threshold = .0001)

## SVM

In [ ]:
# convert Y to integer (not float)

bar_direction_int = [int(x/45) for x in bar_direction]
bar_position_int = [int((x+3)/4) for x in bar_position]

plt.plot(bar_position_int)

In [ ]:
# Dimensionality reduction on time series (reduce # of voxels to avoid overfitting)

from sklearn.decomposition import PCA

# number of components for each feature
n_comps_dir = 14
n_comps_pos = 12

# Initializes PCA
pca_model_dir = PCA(n_components = n_comps_dir)
pca_model_pos = PCA(n_components = n_comps_pos)

# Performs PCA
pca_model_dir.fit(time_series_bar1)
pca_model_pos.fit(time_series_bar1)

# get PCA scores
time_scores_bar1_dir = pca_model_dir.transform(time_series_bar1)
time_scores_bar1_pos = pca_model_pos.transform(time_series_bar1)

# check shape
time_scores_bar1_dir.shape

In [ ]:
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split

# Split data into training and  test data:
# data for SVM
X1 = time_scores_bar1_dir
X2 = time_scores_bar1_pos
Y1 = bar_direction_int
Y2 = bar_position_int
# label matrices, 0 is direction data, 1 is position data
XZ1 = np.zeros_like(X1)
XZ2 = np.ones_like(X2)
YZ1 = np.zeros_like(Y1)
YZ2 = np.ones_like(Y2)
# stack everything together for splitting
X = np.append(X1, X2, 1)
Y = np.vstack([Y1, Y2]).T
XZ = np.append(XZ1, XZ2, 1)
YZ = np.vstack([YZ1, YZ2]).T
# split
X_train, X_test, Y_train, Y_test, XZ_train, XZ_test, YZ_train, YZ_test = train_test_split(X,Y,XZ,YZ, test_size = .2) # random state depend on how the data looks like (we might need to shuffle it)
# recover data from stack
X1_train = np.reshape(X_train[XZ_train==0], (-1, n_comps_dir))
X2_train = np.reshape(X_train[XZ_train==1], (-1, n_comps_pos))
Y1_train = Y_train[YZ_train==0]
Y2_train = Y_train[YZ_train==1]
X1_test = np.reshape(X_test[XZ_test==0], (-1, n_comps_dir))
X2_test = np.reshape(X_test[XZ_test==1], (-1, n_comps_pos))
Y1_test = Y_test[YZ_test==0]
Y2_test = Y_test[YZ_test==1]

# Implement SVM:
SVM_classifier_dir = svm.SVC(kernel='rbf', max_iter=1e6, C=1e3) # Type of Kernel can be changed later 'rbf' is a good other option
SVM_classifier_pos = svm.SVC(kernel='rbf', max_iter=1e6, C=1e2)
# train the model
SVM_classifier_dir.fit(X1_train,Y1_train)
SVM_classifier_pos.fit(X2_train,Y2_train)

# predict the response:
predictions_dir = SVM_classifier_dir.predict(X1_test)
predictions_pos = SVM_classifier_pos.predict(X2_test)

In [ ]:
# Evaluation of the model:
predictions = np.vstack([predictions_dir, predictions_pos]).T
#count for how many indices both values are correct
counter = 0
for i in range(np.size(predictions,0)):
  if (predictions[i, 0] == Y_test[i, 0] and predictions[i, 1] == Y_test[i, 1]): counter += 1

print('TOTAL ACCURACY = ' + str(counter/np.size(predictions, 0)))
print(' ')
print('directions:')
print('accuracy', metrics.accuracy_score(Y1_test, predictions_dir))
print(metrics.classification_report(Y1_test, predictions_dir))
confusion_matrix = metrics.confusion_matrix(Y1_test, predictions_dir)
print(confusion_matrix)

print('positions:')
print('accuracy', metrics.accuracy_score(Y2_test, predictions_pos))
print(metrics.classification_report(Y2_test, predictions_pos))
confusion_matrix = metrics.confusion_matrix(Y2_test, predictions_pos)
print(confusion_matrix)


print('combined:')
print('accuracy '+ str(counter/np.size(predictions, 0)))

# Expanding circle (discrete stim)

In [ ]:
## CW STIMULUS MATRIX

# define stimulus cycle
n_before = 22
period = 32
cycles = 8
n_after = 22

import math

# FIND INITIAL ANGLE

# get circle center
center_y = len(stim_cw[0, :, 0]) / 2 # find y-coord of wedge center
center_x = len(stim_cw[0, 0, :]) / 2 # find x-coord of wedge center

# find the x-coord of where the white starts at y = 0
top_row = np.sum(stim_cw, axis = 3)[22, 0, :]
x2 = np.where(top_row == np.amax(top_row))[0][0]

# get angle of right triangle (treat center as (0,0))
v1 = np.array([0, center_y])
v2 = np.array([x2 - center_x, center_y])

inner = np.inner(v1, v2)
norms = np.linalg.norm(v1) * np.linalg.norm(v2)

cos = inner / norms
rad = np.arccos(np.clip(cos, -1.0, 1.0)) # radians

# add 45 degrees (pi/180 rad) to get center
initial_angle = rad + np.deg2rad(45)

In [ ]:
# plot
plt.imshow(stim_cw[22, :, :, :]) # plot initial frame
plt.plot(center_x, center_y, 'ro') # draw point at center
plt.plot([center_x, center_x], [center_y, 0], 'r--') # draw straight vertical line at the x-coord of the center
plt.plot(x2, 0, 'ro') # draw point where wedge begins
plt.plot([center_x, x2], [center_y, 0], 'r--') # draw line from center to where wedge begins

mid_x = 144*np.cos(initial_angle)
mid_y = 144*np.cos(np.deg2rad(180) - initial_angle)

plt.plot(center_x + mid_x, center_y + mid_y, 'ro')
plt.show()
center_y

In [ ]:
import itertools

# discrete stimulus: each angle is an event

total_rad = 2 * np.pi
rate = total_rad / period # rad/s
stim_time = 300 - n_before - n_after

one_cycle = np.arange(start = initial_angle, stop = initial_angle + total_rad, step = rate)

for r in range(len(one_cycle)):
  if one_cycle[r] >= total_rad:
    one_cycle[r] = one_cycle[r] - total_rad

full_stim_cw = np.tile(one_cycle, 8)
full_stim_cw = list(itertools.chain([0] * n_before, full_stim_cw.tolist(), [0] * n_after))

plt.plot(full_stim_cw, 'ro')
plt.vlines(np.arange(start = n_before, stop = n_before + stim_time, step = period), ymin = 0, ymax = total_rad, linestyles = "--")
# one_cycle

In [ ]:
# create design matrix

d_cw = {"time": range(300),
    "event": full_stim_cw
     }

df_design_mat_cw = pd.DataFrame(d_cw)
df_design_mat_cw = pd.get_dummies(df_design_mat_cw["event"])
design_mat_cw = df_design_mat_cw.to_numpy()

dm_cw = Design_Matrix(design_mat_cw,
              sampling_freq = 1,
              columns = df_design_mat_cw.columns)

dm_cw.heatmap()

# dm_with_cosine = dm.add_dct_basis(duration = 20)
# dm_with_cosine.heatmap()

In [ ]:
# Convolve each angle event with HRF

padding = len(hrf_signal) - 1
padded_stim_cw = np.concatenate([np.zeros((padding, design_mat_cw.shape[1])), design_mat_cw])

design_mat_convolved_cw= np.zeros(padded_stim_cw.shape)
for i in range(design_mat_cw.shape[1]):
  design_mat_convolved_cw[:, i] = np.convolve(design_mat_cw[:, i], hrf_signal)

In [ ]:
# GLM MODEL

constant = np.ones(padded_stim_cw.shape[0])
X_cw = np.column_stack([constant, design_mat_convolved_cw])[n_before:(300 - n_after)]

theta_hats_cw = np.linalg.inv(X_cw.T @ X_cw) @ X_cw.T @ time_series_cw[n_before:(300 - n_after)]

predicted_time_series_cw = X_cw @ theta_hats_cw

In [ ]:
# PLOT THETA WEIGHTS ON BRAIN

filename = "Q1-Q6_R440.L.inflated.32k_fs_LR.surf.gii"
pathf = os.path.join(DATA_DIR, fname_ex, "surf", filename)
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(theta_hats_cw[1, :], cifti_cw,'LEFT'),
                   cmap = 'hsv', threshold = .0001)

**TO IMPROVE:**

**GLM model: Compressive spatial summation in the human visual cortex**

Kay et al., 2013

**accounts for nonlinear effect in spatial summation*

"The GLM analysis deconvolves an HRF from the raw time-series data of each voxel to estimate the voxel response amplitude to each aperature; the pRF analyses are the outputs of the GLM analysis rather than the raw time-series data"

* voxel-specific HRF can be estimated directly from the data (finite impulse response - FIR; Dale, 1999)
* low-frequency fluctuation - use polynomials as regressors for the baseline signal level
* reduce overfitting of FIR model by incorporating time-event separability

**to consider if we want to improve our HRF modeling (instead of just using canonical for all voxels/regions):**

* FIR - BOLD signal assumed to be a linear, time-invariant system with respect to the stimulus
* HDR estimated for each stimulus event type using a set of shifted delta functions as regressors (no assumption on shape of HDR)
* two types of effects: stimulus effects (transient HDRs to stimulus events) & nuisance effects (persistent baseline signal that may vary over time)


* e = number of event types
* I = number of time points in one HDR
* m = number of nuisance terms
* t = number of time-series data points

time-series data modeled as y = Xh + Sb + n
* y = data (t x 1)
* X = stimulus matrix (t x el); concatenation of the stimulus convolution matrix for each event type; consists of shifted versions of a binary sequence, where ones indicate event occurrences
* h = concatenation of the HDR associated with each event type (el x 1)
* S = nuiasnce matrix (t x m)
* b = set of nuisance params (m x 1)
* n = noise term (t x 1)

**Angle of stimulus (ellipse, not circle)**

In [ ]:
# find circle edges
stim_cw_sum = np.sum(stim_cw, axis = 3)

# bin rgb variable
  # white is (R = 255, G = 255, B = 255) (765)
  # black is (R = 0, G = 0, B = 0)

stim_cw_bin = np.zeros(stim_cw_sum.shape)
for i in range(300):
  for h in range(stim_cw_sum.shape[1]):
    stim_cw_bin[i, h, :] = np.digitize(stim_cw_sum[i, h, :], bins = (50, 700))

# find edges (bin = 1)

borders = []
for i in range(300):
    stim_slice = stim_cw_bin[i, :, :]
    X = np.argwhere(stim_slice == 1)

    borders.append(X)

In [ ]:
plt.plot(borders[22][:, 0], borders[22][:, 1], 'ro', alpha = 0.5)
# plt.xlim([0, stim_cw_bin.shape[2]])
# plt.ylim([stim_cw_bin.shape[1], 0])

# EXP/CON ring (discrete stim)

time series data

In [ ]:
# pick a run
run_exp = 'EXP'

# define filename
filename_signal_exp = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_exp}_7T_AP.dtseries.nii')

# load the file with nibabel
cifti_exp = nib.load(filename_signal_exp)

time_series_exp = cifti_exp.get_fdata()

stimulus data

In [ ]:
# define filename
filename_stim_exp = os.path.join(DATA_DIR, fname_ex, f'stims/7T_RET{run_exp}.npy')

# load the numpy array for the stimulus
stim_exp = np.load(filename_stim_exp)

# check the shape
stim_exp.shape # time x Y (height) x X (width) x 3


In [ ]:
# plot stim
plt.imshow(stim_exp[54, :, :, :])

# outer appears at 22s
# outer stops expanding at 44s

# inner appears at 27s
# inner disappears at 50s

define stimulus as discrete event at each time point

In [ ]:
# define stimulus cycle
n_before = 22
period = 32
cycles = 8
n_after = 22
stim_time = 300 - n_before - n_after

# find area at each time point
outer_cycle = np.arange(1, 29)
outer_cycle[22:] = 22
inner_cycle = np.concatenate([[0] * 6, np.arange(1, 23)])

outer_radius = np.zeros(300)
inner_radius = np.zeros(300)

for i in range(8):
  outer_radius[(22 + i*32):(50 + i*32)] = outer_cycle
  inner_radius[(22 + i*32):(50 + i*32)] = inner_cycle

# calculate area and round to nearest integer (for SVM)
stim_area_exp = np.pi*outer_radius**2-np.pi*inner_radius**2
stim_area_exp_int = [int(x) for x in stim_area_exp]

In [ ]:
# plot
plt.plot(stim_area_exp_int, 'ro')
plt.vlines(np.arange(start = n_before, stop = n_before + stim_time, step = period), ymin = 0, ymax = np.max(stim_area_exp_int), linestyles = "--")

create design matrix

In [ ]:
d_exp = {"time": range(300),
    "event": stim_area_exp_int
     }

df_design_mat_exp = pd.DataFrame(d_exp)
df_design_mat_exp = pd.get_dummies(df_design_mat_exp["event"])
design_mat_exp = df_design_mat_exp.to_numpy()

dm_exp = Design_Matrix(design_mat_exp,
              sampling_freq = 1,
              columns = df_design_mat_exp.columns)

dm_exp.heatmap()

# dm_with_cosine = dm.add_dct_basis(duration = 20)
# dm_with_cosine.heatmap()

In [ ]:
# Convolve each area event with HRF

padding = len(hrf_signal) - 1
padded_stim_exp = np.concatenate([np.zeros((padding, design_mat_exp.shape[1])), design_mat_exp])

design_mat_convolved_exp = np.zeros(padded_stim_exp.shape)
for i in range(design_mat_exp.shape[1]):
  design_mat_convolved_exp[:, i] = np.convolve(design_mat_exp[:, i], hrf_signal)

In [ ]:
# GLM MODEL

constant = np.ones(padded_stim_exp.shape[0])
X_exp = np.column_stack([constant, design_mat_convolved_exp])[n_before:(300 - n_after)]

theta_hats_exp = np.linalg.inv(X_exp.T @ X_exp) @ X_exp.T @ time_series_exp[n_before:(300 - n_after)]

predicted_time_series_exp = X_exp @ theta_hats_exp

In [ ]:
# PLOT THETA WEIGHTS ON BRAIN

filename = "Q1-Q6_R440.L.inflated.32k_fs_LR.surf.gii"
pathf = os.path.join(DATA_DIR, fname_ex, "surf", filename)
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(theta_hats_exp[1, :], cifti_exp,'LEFT'),
                   cmap = 'hsv', threshold = .0001)

CON (EXP reversed)

In [ ]:
# get time series for contracting ring

# pick a run
run_con = 'CON'

# define filename
filename_signal_con = os.path.join(DATA_DIR, fname_ex, f'tfMRI_RET{run_con}_7T_PA.dtseries.nii')

# load the file with nibabel
cifti_con = nib.load(filename_signal_con)

# extract time series data
time_series_con = cifti_con.get_fdata()

In [ ]:
# reverse expanding ring to get contracting ring
stim_area_con_int = stim_area_exp_int[::-1]

In [ ]:
# plot
plt.plot(stim_area_con_int, 'ro')
plt.vlines(np.arange(start = n_before, stop = n_before + stim_time, step = period), ymin = 0, ymax = np.max(stim_area_con_int), linestyles = "--")

In [ ]:
# design matrix

d_con = {"time": range(300),
    "event": stim_area_con_int
     }

df_design_mat_con = pd.DataFrame(d_con)
df_design_mat_con = pd.get_dummies(df_design_mat_con["event"])
design_mat_con = df_design_mat_con.to_numpy()

dm_con = Design_Matrix(design_mat_con,
              sampling_freq = 1,
              columns = df_design_mat_con.columns)

dm_con.heatmap()

In [ ]:
# Convolve each area event with HRF

padding = len(hrf_signal) - 1
padded_stim_con = np.concatenate([np.zeros((padding, design_mat_con.shape[1])), design_mat_con])

design_mat_convolved_con = np.zeros(padded_stim_con.shape)
for i in range(design_mat_con.shape[1]):
  design_mat_convolved_con[:, i] = np.convolve(design_mat_con[:, i], hrf_signal)

In [ ]:
# GLM MODEL

constant = np.ones(padded_stim_con.shape[0])
X_con = np.column_stack([constant, design_mat_convolved_con])[n_before:(300 - n_after)]

theta_hats_con = np.linalg.inv(X_con.T @ X_con) @ X_con.T @ time_series_con[n_before:(300 - n_after)]

predicted_time_series_con = X_con @ theta_hats_con

In [ ]:
# PLOT THETA WEIGHTS ON BRAIN
filename = "Q1-Q6_R440.L.inflated.32k_fs_LR.surf.gii"
pathf = os.path.join(DATA_DIR, fname_ex, "surf", filename)
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(theta_hats_con[1, :], cifti_con,'LEFT'),
                   cmap = 'hsv', threshold = .0001)

T-contrast

In [ ]:
# WHERE IS EXP MORE INTERESTING THAN CON?? (T-CONTRAST)
cope_1_neg1 = 1 * theta_hats_exp + -1 * theta_hats_con
t_1_neg1 = cope_1_neg1/np.std(cope_1_neg1)

# WHERE IS CON MORE INTERESTING THAN EXP?? (T-CONTRAST)
cope_neg1_1 = -1 * theta_hats_exp + 1 * theta_hats_con
t_neg1_1 = cope_neg1_1/np.std(cope_neg1_1)

In [ ]:
# PLOT T-CONTRASTS WEIGHTS ON BRAIN

filename = "Q1-Q6_R440.L.inflated.32k_fs_LR.surf.gii"
pathf = os.path.join(DATA_DIR, fname_ex, "surf", filename)

# EXP > CON
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(t_1_neg1[1, :], cifti_con,'LEFT'),
                   cmap = 'hsv', threshold = .0001)

In [ ]:
# CON > EXP
plotting.view_surf(surf_mesh = pathf,
                   surf_map = surf_data_from_cifti(t_neg1_1[1, :], cifti_con,'LEFT'),
                   cmap = 'hsv', threshold = .0001)

# SVM (right now for EXP)

In [ ]:
# Dimensionality reduction on time series (reduce # of voxels to avoid overfitting)

from sklearn.decomposition import PCA

# Initializes PCA
pca_model = PCA(n_components = 2)

# Performs PCA
pca_model.fit(time_series_exp)

# get PCA scores
time_scores_exp = pca_model.transform(time_series_exp)

# check shape
time_scores_exp.shape

In [ ]:
# plot PCA of timepoints

plt.figure()
cmap = plt.cm.get_cmap('hsv')
plt.scatter(x=time_scores_exp[:, 0], y=time_scores_exp[:, 1], c=stim_area_exp_int, cmap=cmap)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.colorbar(ticks=np.arange(0, np.max(stim_area_exp_int) + 1, 100))
plt.clim(-0.5, np.max(stim_area_exp_int) + 1)

In [ ]:
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split

# Split data into training and test data:
X = time_scores_exp
Y = stim_area_exp_int
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2 , random_state = 209) # random state depend on how the data looks like (we might need to shuffle it)

# Implement SVM:
SVM_classifier = svm.SVC(kernel='rbf') # Type of Kernel can be changed later 'rbf' is a good other option

# train the model
SVM_classifier.fit(X_train,Y_train)

# predict the response:
predictions = SVM_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# Evaluation of the model:

print(metrics.classification_report(Y_test, predictions))
confusion_matrix = metrics.confusion_matrix(Y_test, predictions)
# print(confusion_matrix)

# plot conf mat
ConfusionMatrixDisplay.from_predictions(Y_test, predictions)

How many PCs are needed for SVM?

In [ ]:
# Evaluate the accuracy of EXP SVM using top n components

np.random.seed(2022)

n = np.arange(1, 21) # 8 min runtime
accuracy = []

for i in n:
  print("running simulation for ", str(i), "components")
  acc = run_svm_sims(i)
  accuracy.append(acc)

In [ ]:
# Visualize evaluations
plt.plot(n, accuracy, 'ko')
plt.ylim([0, 1])

# Basic CNN

In [ ]:
# Imports

import torch
from torch import nn
from torch import optim
import matplotlib as mpl

In [ ]:
# @title define convolutional layer

class ConvolutionalLayer(nn.Module):
  """Deep network with one convolutional layer
     Attributes: conv (nn.Conv2d): convolutional layer
  """
  def __init__(self, c_in=3, c_out=6, K=7, filters=None):
    """Initialize layer

    Args:
        c_in: number of input stimulus channels
        c_out: number of output convolutional channels
        K: size of each convolutional filter
        filters: (optional) initialize the convolutional weights

    """
    super().__init__()
    self.conv = nn.Conv2d(c_in, c_out, kernel_size=K,
                          padding=K//2, stride=1)
    if filters is not None:
      self.conv.weight = nn.Parameter(filters)
      self.conv.bias = nn.Parameter(torch.zeros((c_out,), dtype=torch.float32))

  def forward(self, s):
    """Run stimulus through convolutional layer

    Args:
        s (torch.Tensor): n_stimuli x c_in x h x w tensor with stimuli

    Returns:
        (torch.Tensor): n_stimuli x c_out x h x w tensor with convolutional layer unit activations.

    """
    a = self.conv(s)  # output of convolutional layer

    return a

In [ ]:
# @title Helper function: define filters

def filters(out_channels=6, K=7):
  """ make example filters, some center-surround and gabors
  Returns:
      filters: out_channels x K x K
  """
  grid = np.linspace(-K/2, K/2, K).astype(np.float32)
  xx,yy = np.meshgrid(grid, grid, indexing='ij')

  # create center-surround filters
  sigma = 1.1
  gaussian = np.exp(-(xx**2 + yy**2)**0.5/(2*sigma**2))
  wide_gaussian = np.exp(-(xx**2 + yy**2)**0.5/(2*(sigma*2)**2))
  center_surround = gaussian - 0.5 * wide_gaussian

  # create gabor filters
  thetas = np.linspace(0, 180, out_channels-2+1)[:-1] * np.pi/180
  gabors = np.zeros((len(thetas), K, K), np.float32)
  lam = 10
  phi = np.pi/2
  gaussian = np.exp(-(xx**2 + yy**2)**0.5/(2*(sigma*0.4)**2))
  for i,theta in enumerate(thetas):
    x = xx*np.cos(theta) + yy*np.sin(theta)
    gabors[i] = gaussian * np.cos(2*np.pi*x/lam + phi)

  filters = np.concatenate((center_surround[np.newaxis,:,:],
                            -1*center_surround[np.newaxis,:,:],
                            gabors),
                           axis=0)
  filters /= np.abs(filters).max(axis=(1,2))[:,np.newaxis,np.newaxis]
  filters -= filters.mean(axis=(1,2))[:,np.newaxis,np.newaxis]
  # convert to torch
  filters = torch.from_numpy(filters)
  # add channel axis
  filters = filters.unsqueeze(1)

  return filters

In [ ]:
# Create and visualize filters

example_filters = filters(out_channels=6, K=7)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(example_filters[0,0], vmin=-1, vmax=1, cmap='bwr')
plt.title('center-surround filter')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(example_filters[4,0], vmin=-1, vmax=1, cmap='bwr')
plt.title('gabor filter')
plt.axis('off');

Run convolutional layer on stimulus

In [ ]:
stim_exp_sum = np.sum(stim_exp, axis = 3)
stim_exp_sum.shape

In [ ]:
# Stimulus parameters

in_channels = 1 # how many input channels in our images
h = stim_exp.shape[1] # height of images
w = stim_exp.shape[2] # width of images

# Convolution layer parameters
K = 7 # filter size
out_channels = 6 # how many convolutional channels to have in our layer
example_filters = filters(out_channels, K) # create filters to use

convout = np.zeros(0) # assign convolutional activations to convout

# Initialize conv layer and add weights from function filters
# you need to specify :
# * the number of input channels c_in
# * the number of output channels c_out
# * the filter size K
convLayer = ConvolutionalLayer(c_in = in_channels,
                               c_out = out_channels,
                               K = K,
                               filters = example_filters)

# run convLayer on stimulus
stimuli_exp = torch.zeros((len(stim_area_exp_int), in_channels, h, w), dtype = torch.float32)

## 3 channels?
# for i in range(len(stim_area_exp_int)):
#
#  for c in range(in_channels):
#
#    stimuli_exp[i, c] = torch.tensor(stim_exp[i, :, :, c], dtype = torch.float32)

## 1 channel (sum of 3 channels)
for i in range(len(stim_area_exp_int)):

  a = stim_exp_sum[i, :, :]
  a = np.vstack(a).astype(float)
  stimuli_exp[i, 0] = torch.tensor(torch.from_numpy(a), dtype = torch.float32)

convout = convLayer(stimuli_exp).detach() # detach gradients

In [ ]:
# @title plot activations

def show_stimulus(img, ax=None):
  """Visualize a stimulus"""
  if ax is None:
    ax = plt.gca()
  ax.imshow(img, cmap=mpl.cm.binary)
  ax.set_aspect('auto')
  ax.set_xticks([])
  ax.set_yticks([])
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)

def plot_example_activations(stimuli, act, channels=[0]):
  """ plot activations act and corresponding stimulus
  Args:
        stimuli: stimulus input to convolutional layer (n x h x w) or (h x w)
        act: activations of convolutional layer (n_bins x conv_channels x n_bins)
        channels: which conv channels to plot
  """
  if stimuli.ndim>2:
    n_stimuli = stimuli.shape[0]
  else:
    stimuli = stimuli.unsqueeze(0)
    n_stimuli = 1

  fig, axs = plt.subplots(n_stimuli,1+len(channels),figsize=(12,12))

  # plot stimulus
  for i in range(n_stimuli):
    show_stimulus(stimuli[i].squeeze(), ax=axs[i, 0])
    axs[i, 0].set_title('stimulus')

    # plot example activations
    for k, (channel, ax) in enumerate(zip(channels, axs[i][1:])):
      img=ax.imshow(act[i,channel], vmin=-3, vmax=3, cmap='bwr')
      ax.set_xlabel('x-pos')
      ax.set_ylabel('y-pos')
      ax.set_title('channel %d'%channel)
  ax = fig.add_axes([1.05,0.8,0.01,0.1])
  plt.colorbar(img, cax=ax)
  ax.set_title('activation\n strength')

In [ ]:
plot_example_activations(stimuli_exp[40:46, :, :, :], convout[40:46, :, :, :], channels=np.arange(0, out_channels))

# Normative encoding model

In [ ]:
# @title Helper functions

class CNN(nn.Module):
  """Deep convolutional network with one convolutional + pooling layer followed
  by one fully connected layer

  Args:
    h_in (int): height of input image, in pixels (i.e. number of rows)
    w_in (int): width of input image, in pixels (i.e. number of columns)

  Attributes:
    conv (nn.Conv2d): filter weights of convolutional layer
    pool (nn.MaxPool2d): max pooling layer
    dims (tuple of ints): dimensions of output from pool layer
    fc (nn.Linear): weights and biases of fully connected layer
    out (nn.Linear): weights and biases of output layer

  """

  def __init__(self, h_in, w_in):
    super().__init__()
    C_in = 1  # input stimuli have only 1 input channel
    C_out = 6  # number of output channels (i.e. of convolutional kernels to convolve the input with)
    K = 7  # size of each convolutional kernel
    Kpool = 8  # size of patches over which to pool
    self.conv = nn.Conv2d(C_in, C_out, kernel_size=K, padding=K//2)  # add padding to ensure that each channel has same dimensionality as input
    self.pool = nn.MaxPool2d(Kpool)
    self.dims = (C_out, h_in // Kpool, w_in // Kpool)  # dimensions of pool layer output
    self.fc = nn.Linear(np.prod(self.dims), 10)  # flattened pool output --> 10D representation
    self.out = nn.Linear(10, 1)  # 10D representation --> scalar
    self.conv.weight = nn.Parameter(filters(C_out, K))
    self.conv.bias = nn.Parameter(torch.zeros((C_out,), dtype=torch.float32))

  def forward(self, x):
    """Classify grating stimulus as tilted right or left

    Args:
      x (torch.Tensor): p x 48 x 64 tensor with pixel grayscale values for
          each of p stimulus images.

    Returns:
      torch.Tensor: p x 1 tensor with network outputs for each input provided
          in x. Each output should be interpreted as the probability of the
          corresponding stimulus being tilted right.

    """
    x = x.unsqueeze(1)  # p x 1 x 48 x 64, add a singleton dimension for the single stimulus channel
    x = torch.relu(self.conv(x))  # output of convolutional layer
    x = self.pool(x)  # output of pooling layer
    x = x.view(-1, np.prod(self.dims))  # flatten pooling layer outputs into a vector
    x = torch.relu(self.fc(x))  # output of fully connected layer
    x = torch.sigmoid(self.out(x))  # network output
    return x


def train(net, train_data, train_labels,
          n_epochs=25, learning_rate=0.0005,
          batch_size=100, momentum=.99):
  """Run stochastic gradient descent on binary cross-entropy loss for a given
  deep network (cf. appendix for details)

  Args:
    net (nn.Module): deep network whose parameters to optimize with SGD
    train_data (torch.Tensor): n_train x h x w tensor with stimulus gratings
    train_labels (torch.Tensor): n_train x 1 tensor with true tilt of each
      stimulus grating in train_data, i.e. 1. for right, 0. for left
    n_epochs (int): number of times to run SGD through whole training data set
    batch_size (int): number of training data samples in each mini-batch
    learning_rate (float): learning rate to use for SGD updates
    momentum (float): momentum parameter for SGD updates

  """

  # Initialize binary cross-entropy loss function
  loss_fn = nn.BCELoss()

  # Initialize SGD optimizer with momentum
  optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

  # Placeholder to save loss at each iteration
  track_loss = []

  # Loop over epochs
  for i in range(n_epochs):

    # Split up training data into random non-overlapping mini-batches
    ishuffle = torch.randperm(train_data.shape[0])  # random ordering of training data
    minibatch_data = torch.split(train_data[ishuffle], batch_size)  # split train_data into minibatches
    minibatch_labels = torch.split(train_labels[ishuffle], batch_size)  # split train_labels into minibatches

    # Loop over mini-batches
    for stimuli, tilt in zip(minibatch_data, minibatch_labels):

      # Evaluate loss and update network weights
      out = net(stimuli)  # predicted probability of tilt right
      loss = loss_fn(out, tilt)  # evaluate loss
      optimizer.zero_grad()  # clear gradients
      loss.backward()  # compute gradients
      optimizer.step()  # update weights

      # Keep track of loss at each iteration
      track_loss.append(loss.item())

    # Track progress
    if (i + 1) % (n_epochs // 5) == 0:
      print(f'epoch {i + 1} | loss on last mini-batch: {loss.item(): .2e}')

  print('training done!')


def get_hidden_activity(net, stimuli, layer_labels):
  """Retrieve internal representations of network

  Args:
    net (nn.Module): deep network
    stimuli (torch.Tensor): p x 48 x 64 tensor with stimuli for which to
      compute and retrieve internal representations
    layer_labels (list): list of strings with labels of each layer for which
      to return its internal representations

  Returns:
    dict: internal representations at each layer of the network, in
      numpy arrays. The keys of this dict are the strings in layer_labels.

  """

  # Placeholder
  hidden_activity = {}

  # Attach 'hooks' to each layer of the network to store hidden
  # representations in hidden_activity
  def hook(module, input, output):
    module_label = list(net._modules.keys())[np.argwhere([module == m for m in net._modules.values()])[0, 0]]
    if module_label in layer_labels:  # ignore output layer
      hidden_activity[module_label] = output.view(stimuli.shape[0], -1).detach().numpy()
  hooks = [layer.register_forward_hook(hook) for layer in net.children()]

  # Run stimuli through the network
  pred = net(stimuli)

  # Remove the hooks
  [h.remove() for h in hooks]

  return hidden_activity

In [ ]:
# @title Plotting functions

def plot_corr_matrix(rdm, ax=None):
  """Plot dissimilarity matrix

  Args:
    rdm (numpy array): n_stimuli x n_stimuli representational dissimilarity
      matrix
    ax (matplotlib axes): axes onto which to plot

  Returns:
    nothing

  """
  if ax is None:
    ax = plt.gca()
  image = ax.imshow(rdm, vmin=0.0, vmax=2.0)
  ax.set_xticks([])
  ax.set_yticks([])
  cbar = plt.colorbar(image, ax=ax, label='dissimilarity')


def plot_multiple_rdm(rdm_dict):
  """Draw multiple subplots for each RDM in rdm_dict."""
  fig, axs = plt.subplots(1, len(rdm_dict),
                          figsize=(4 * len(resp_dict), 3.5))

  # Compute RDM's for each set of responses and plot
  for i, (label, rdm) in enumerate(rdm_dict.items()):

    image = plot_corr_matrix(rdm, axs[i])
    axs[i].set_title(label)


def plot_rdm_rdm_correlations(rdm_sim):
  """Draw a bar plot showing between-RDM correlations."""
  f, ax = plt.subplots()
  ax.bar(rdm_sim.keys(), rdm_sim.values())
  ax.set_xlabel('Deep network model layer')
  ax.set_ylabel('Correlation of model layer RDM\nwith mouse V1 RDM')


def plot_rdm_rows(ori_list, rdm_dict, rdm_oris):
  """Plot the dissimilarity of response to each stimulus with response to one
  specific stimulus

  Args:
    ori_list (list of float): plot dissimilarity with response to stimulus with
      orientations closest to each value in this list
    rdm_dict (dict): RDM's from which to extract dissimilarities
    rdm_oris (np.ndarray): orientations corresponding to each row/column of RDMs
    in rdm_dict

  """
  n_col = len(ori_list)
  f, axs = plt.subplots(1, n_col, figsize=(4 * n_col, 4), sharey=True)

  # Get index of orientation closest to ori_plot
  for ax, ori_plot in zip(axs, ori_list):
    iori = np.argmin(np.abs(rdm_oris - ori_plot))

    # Plot dissimilarity curves in each RDM
    for label, rdm in rdm_dict.items():
      ax.plot(rdm_oris, rdm[iori, :], label=label)

    # Draw vertical line at stimulus we are plotting dissimilarity w.r.t.
    ax.axvline(rdm_oris[iori], color=".7", zorder=-1)

    # Label axes
    ax.set_title(f'Dissimilarity with response\nto {ori_plot: .0f}$^o$ stimulus')
    ax.set_xlabel('Stimulus orientation ($^o$)')

  axs[0].set_ylabel('Dissimilarity')
  axs[-1].legend(loc="upper left", bbox_to_anchor=(1, 1))

In [ ]:
# Stimulus parameters

in_channels = 1 # how many input channels in our images
h = stim_exp.shape[1] # height of images
w = stim_exp.shape[2] # width of images

# Convolution layer parameters
K = 7 # filter size
out_channels = 6 # how many convolutional channels to have in our layer
example_filters = filters(out_channels, K) # create filters to use

# run convLayer on stimulus
stimuli_exp = torch.zeros((len(stim_area_exp_int), h, w), dtype = torch.float32)

## 1 channel (sum of 3 channels)
for i in range(len(stim_area_exp_int)):

  a = stim_exp_sum[i, :, :]
  a = np.vstack(a).astype(float)
  stimuli_exp[i] = torch.tensor(torch.from_numpy(a), dtype = torch.float32)

In [ ]:
# Set random seeds for reproducibility
np.random.seed(2022)
torch.manual_seed(2022)

# Initialize CNN model
net = CNN(h, w)

# stimulus area
area_exp = torch.tensor(stim_area_exp).type(torch.float).unsqueeze(-1)

# Train model
train(net, stimuli_exp, area_exp)

In [ ]:
# extract model internal representations of each stimulus in the V1 data
layer_labels = ['pool', 'fc']
exp_model = get_hidden_activity(net, stimuli_exp, layer_labels)

In [ ]:
print(exp_model['pool'].shape, exp_model['fc'].shape)

In [ ]:
## COMPUTE RDMS FOR THE NEURAL SIGNALS AND EACH LAYER

from scipy.stats import zscore

# z-score responses to each stimulus
z_signal_exp = zscore(time_series_exp, axis=1)
z_pool_exp = zscore(exp_model['pool'], axis=1)
z_fc_exp = zscore(exp_model['fc'], axis=1)

# Compute RDM
RDM_signal_exp = 1 - (1/z_signal_exp.shape[1]) * (np.matmul(z_signal_exp, z_signal_exp.T))
RDM_pool_exp = 1 - (1/z_pool_exp.shape[1]) * (np.matmul(z_pool_exp, z_pool_exp.T))
RDM_fc_exp = 1 - (1/z_fc_exp.shape[1]) * (np.matmul(z_fc_exp, z_fc_exp.T))

In [ ]:
# plot RDMs
plt.rcParams['figure.figsize'] = [20, 15]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

ax1.set_title("fMRI voxels")
ax1.imshow(RDM_signal_exp)

ax2.set_title("Pool layer")
ax2.imshow(RDM_pool_exp)

ax3.set_title("FC layer")
ax3.imshow(RDM_fc_exp)
# cbar = plt.colorbar(fig, ax=ax3, label='dissimilarity')

plt.show()

In [ ]:
# indices of off-diagonal elements
ioffdiag = np.triu_indices(RDM_signal_exp.shape[0], k=1)

# correlation of RDM for FC layer
RDM_signal_exp_offdiag = RDM_signal_exp[ioffdiag]]
RDM_fc_exp_offdiag = RDM_fc_exp[ioffdiag]

np.corrcoef(RDM_signal_exp_offdiag, RDM_fc_exp_offdiag)[0,1]

In [ ]:
# correlation of RDM for pool layer

RDM_pool_exp_offdiag = RDM_pool_exp[ioffdiag]

# find and remove NAs
na_values = np.isnan(RDM_pool_exp_offdiag)

RDM_pool_exp_offdiag_nona = RDM_pool_exp_offdiag[~na_values]
RDM_signal_exp_offdiag_nona = RDM_signal_exp_offdiag[~na_values]

# correlate
np.corrcoef(RDM_signal_exp_offdiag_nona, RDM_pool_exp_offdiag_nona)[0,1]

In [ ]:
z_fc_exp.shape

In [ ]:
# Visualize tuning curves

plt.rcParams['figure.figsize'] = [20, 5]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

ax1.set_title("fMRI response")
voxels = np.random.choice(time_series_exp.shape[1], 3, replace = False)
ax1.plot(time_series_exp[:, voxels])

ax2.set_title("Pool layer respose")
pool_neurons = np.random.choice(exp_model['pool'].shape[1], 3, replace = False)
ax2.plot(exp_model['pool'][:, pool_neurons])

ax3.set_title("FC layer respose")
fc_neurons = np.random.choice(exp_model['fc'].shape[1], 3, replace = False)
ax3.plot(exp_model['fc'][:, fc_neurons])

plt.show()

In [ ]:
# Dimensionality reduction of responses

# SVM for stimulus type

In [ ]:
# Dimensionality reduction on time series (reduce # of voxels to avoid overfitting)

# number of components for each feature
n_comps = 20

# Initializes PCA
pca_model = PCA(n_components = n_comps)

# Performs PCA
pca_model.fit(time_series_stims)

# get PCA scores
time_scores_stims = pca_model.transform(time_series_stims)

# check shape
time_scores_stims.shape

In [ ]:
X = time_scores_stims
Y = stims_on

# split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size = .2) # random state depend on how the data looks like (we might need to shuffle it)
# Implement SVM:
SVM_classifier_stims = svm.SVC(kernel='rbf', max_iter=1e5, C=1e4) # Type of Kernel can be changed later 'rbf' is a good other option
# train the model
SVM_classifier_stims.fit(X_train,Y_train)

# predict the response:
predictions_stims = SVM_classifier_stims.predict(X_test)

In [ ]:
print('accuracy', metrics.accuracy_score(Y_test, predictions_stims))
print(metrics.classification_report(Y_test, predictions_stims))
confusion_matrix = metrics.confusion_matrix(Y_test, predictions_stims)
print(confusion_matrix)

**Stack PCAs of different stimuli:**

In [ ]:
# Stacking time scores and stimulus features for all the stimuli:
# 1 = exp
general_time_scores = np.array(time_scores_exp)
stim_on_exp = stim_area_exp
stim_on_exp[stim_on_exp>0] = 1
stims_on = stim_on_exp
# 2 = con
general_time_scores = np.append(general_time_scores, np.array(time_scores_con), 0)
stim_on_con = stim_area_con
stim_on_con[stim_on_con>0] = 2
stims_on.append(stim_on_con)
# 3 = CW
general_time_scores = np.append(general_time_scores, np.array(time_scores_cw), 0)
stim_on_cw = [np.zeros(22),np.ones(256)*3,np.zeros(22)]
stims_on = np.append(stims_on, stim_on_cw)
# 4 = BAR1
general_time_scores.append(time_scores_bar1)
general_time_scores = np.append(general_time_scores, np.array(time_scores_bar1), 0)
stim_on_bar1 = bar_direction
stim_on_bar1[stim_on_bar1>0] = 4
stims_on = np.append(stims_on, stim_on_bar1)
# 5 = CCW
# 6 = BAR2

In [ ]:
# SVM on the stacked PCAs and stimuli features:

# defining X and Y:
X = general_time_scores
Y = stims_on
# split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size = .2) # random state depend on how the data looks like (we might need to shuffle it)
# Implement SVM:
SVM_classifier_stims = svm.SVC(kernel='rbf', max_iter=1e5, C=1e4) # Type of Kernel can be changed later 'rbf' is a good other option
# train the model
SVM_classifier_stims.fit(X_train,Y_train)
# predict the response:
predictions_stims = SVM_classifier_stims.predict(X_test)

In [ ]:
# Evaluation of the model:
print('accuracy', metrics.accuracy_score(Y_test, predictions_stims))
print(metrics.classification_report(Y_test, predictions_stims))
confusion_matrix = metrics.confusion_matrix(Y_test, predictions_stims)
print(confusion_matrix)